#### 计算两张图片之间的相似度

原理：利用VGG16网络抽像出来的特征做相似度比较。提一下，这里用到的特征是指block5_pool层所抽象的特征，也即是最后一个池化层。在keras提供的VGG16网络api中很容易做到这一点，只需要在初始化VGG16网络的时候设置include_top=False，就能丢弃掉最后的全连接层，最后的输出层便是最后一个池化层。

In [9]:
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image


import ssl
ssl._create_default_https_context = ssl._create_unverified_context # 下载模型的时候不想进行ssl证书校验

!export TF_CPP_MIN_LOG_LEVEL=2  # 屏蔽警告信息 （这是在jupyter中执行的shell命令，可以去掉这行不影响结果）

# 初始化VGG16模型 include_top=False 是指不保留顶层的3个全连接网络层
model = VGG16(weights='imagenet', include_top=False)

def get_feature(path):
    """
    VGG16模型抽取图片特征
    """
    # 图片转成PIL的Image对象，并且对图片做了缩放
    img = image.load_img(path, target_size=(224, 224))

    # 图片转成矩阵、并扩充了维度、最后是预处理
    predict_img = preprocess_input(np.expand_dims(image.img_to_array(img),0))

    # 丢入vgg16网络做特征抽取，最后返回特征并展平成一维向量方便计算余弦相似度
    return model.predict(predict_img).flatten()

def cos_sim(a, b):
    """
    计算两个向量之间的余弦相似度
    """
    a = np.mat(a)
    b = np.mat(b)
    return float(a * b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

Using TensorFlow backend.


In [19]:
image_path = ['/Users/liuliangjun/Downloads/1.jpg', '/Users/liuliangjun/Downloads/2.jpg', '/Users/liuliangjun/Downloads/3.jpg']
ft1, ft2, ft3 = [get_feature(p) for p in image_path]

In [20]:
# 图1、图2相似度
cos_sim(ft1, ft2)

0.9247887575725511

In [21]:
# 图2、图3相似度
cos_sim(ft2, ft3)

0.9832494987242002

In [22]:
# 图1、图3相似度
cos_sim(ft1, ft3)

0.9130592505927942

In [ ]:
# 找出最相似的一组图片
from glob import glob
from itertools import combinations

image_path = '/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/train/cat.*'

max_sim = 0
p1 = p2 = None

for i,j in combinations(glob(image_path)[:5], 2):
    sim = cos_sim(get_feature(i), get_feature(j))
    if max_sim < sim:
        max_sim = sim
        p1, p2 = i, j
        
print(max_sim, p1, p2)

In [ ]:
image.load_img(p1, target_size=(224, 224))

In [ ]:
image.load_img(p2, target_size=(224, 224))

#### 尝试用自己构造的CNN网络抽取特征

不训练直接根据特征计算相似度、也能比较出来、但是相似度不对

In [13]:
new_model = keras.models.load_model('cats.h5')

new_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 64)        36928     
______________________________________________________

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def image_preporcess(img_path, target_size):
    # 以灰度图的形式读入图片
    image = cv2.imread(img_path, 0)
    
    # resize 尺寸
    ih, iw = target_size
    # 原始图片尺寸
    h,  w = image.shape

    # 计算缩放后图片尺寸
    scale = min(iw/w, ih/h)
    nw, nh = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    # 创建一张画布，画布的尺寸就是目标尺寸
    # fill_value=120为灰色画布
    image_paded = np.full(shape=[ih, iw], fill_value=120)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2

    # 将缩放后的图片放在画布中央
    image_paded[dh:nh+dh, dw:nw+dw] = image_resized
    
    # 归一化处理
    image_paded = image_paded / 255.

    return image_paded

if __name__=="__main__":
    # TODO cv2.imread(image_path, 0) 读取灰度图像
    image_path = "/Users/liuliangjun/PycharmProjects/vance/images/dogAndCat/train/cat.0.jpg"
    img=image_preporcess(image_path,(100,100))
    plt.imshow(img)

In [14]:
image_path = ['/Users/liuliangjun/Downloads/1.jpg', '/Users/liuliangjun/Downloads/2.jpg', '/Users/liuliangjun/Downloads/3.jpg']
img = image_preporcess(image_path[0], (100, 100))

# 图片转成矩阵、并扩充了维度、最后是预处理
predict_img = np.expand_dims(img,0)
ft1 = new_model.predict(predict_img)

In [18]:
help(new_model.predict)

Help on method predict in module tensorflow.python.keras.engine.training:

predict(x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Generates output predictions for the input samples.
    
    Computation is done in batches.
    
    Arguments:
        x: Input samples. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A `tf.data` dataset or a dataset iterator.
          - A generator or `keras.utils.Sequence` instance.
        batch_size: Integer or `None`.
            Number of samples per gradient update.
            If unspecified, `batch_size` will default to 32.
            Do not specify the `batch_size` is your data is in the
            form 